In [ ]:
!pip install ultralytics
from ultralytics import YOLO

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.4/799.4 kB 9.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
!unzip drive/MyDrive/CrowdHuman.zip -d dataset/

Streaming output truncated to the last 5000 lines.
  inflating: dataset/CrowdHuman/labels/train/273278-f2726000dc9c49fa_jpg.rf.ccb092e8c2f9b2901a83e0dd675d3a31.txt  
  inflating: dataset/CrowdHuman/labels/train/273278-f27c0001e66d98f_jpg.rf.1681f9b3bb2cf6b0eba3aa693936e1e6.txt  
  inflating: dataset/CrowdHuman/labels/train/273278-f28200001224f875_jpg.rf.d790a98aa8b4a79f820f833d1f88a60a.txt  
  inflating: dataset/CrowdHuman/labels/train/273278-f2d81000016401c2_jpg.rf.0979d7648cfc266bfde4f397c56cd52e.txt  
  inflating: dataset/CrowdHuman/labels/train/273278-f2e0a00045dc2b8f_jpg.rf.5564f59df9dbbf08eaf2f1435dde0449.txt  
  inflating: dataset/CrowdHuman/labels/train/273278-f2e8400026f60d9d_jpg.rf.d9db8f50d7c3b75295f2c3f5d83ce8e5.txt  
  inflating: dataset/CrowdHuman/labels/train/273278-f3055000500e010e_jpg.rf.9b79ea1f5603ca5a6b86d282e7a5fda6.txt  
  inflating: dataset/CrowdHuman/labels/train/273278-f3186000eb5ab4c2_jpg.rf.be40ba14709da134dc41573fb8a6ddd2.txt  
  inflating: dataset/CrowdHuma

In [ ]:
!yolo task=detect mode=train epochs=10 data="/content/dataset/CrowdHuman/data.yaml" model=yolov8s.pt imgsz=640 batch=16 single_cls=true

Ultralytics YOLOv8.2.50 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/dataset/CrowdHuman/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=True, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=

In [ ]:
!pip install ultralytics
!pip install deep-sort-realtime
!pip install opencv-python-headless
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import random
import time
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

In [ ]:

import cv2
import numpy as np
import random
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from google.colab.patches import cv2_imshow

# Load the YOLO model
model = YOLO("bestv8.pt")

# Initialize DeepSORT
deepsort = DeepSort(max_age=15, n_init=3, nms_max_overlap=1.0, max_iou_distance=0.7)

def get_random_color():
    return [random.randint(0, 255) for _ in range(3)]

def process_frame(frame, frame_count, detections, track_colors, threshold, warning_shown):
    if frame_count % 3 == 0:
        results = model(frame, conf=0.3, iou=0.5)
        detections.clear()
        for result in results:
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                w, h = x2 - x1, y2 - y1
                conf = float(box.conf[0])
                if conf > 0.3:
                    detections.append(((x1, y1, w, h), conf))

    tracks = deepsort.update_tracks(detections, frame=frame)
    num_people = 0
    centers = []

    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        bbox = track.to_ltwh()
        x1, y1, w, h = map(int, bbox)
        x2, y2 = x1 + w, y1 + h

        if track_id not in track_colors:
            track_colors[track_id] = get_random_color()

        color = track_colors[track_id]
        center_x, center_y = x1 + w // 2, y1 + h // 2
        centers.append((center_x, center_y))

        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, f'ID: {track_id}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        num_people += 1

    for i, center_i in enumerate(centers):
        for j, center_j in enumerate(centers):
            if i != j and np.linalg.norm(np.array(center_i) - np.array(center_j)) < 50:
                cv2.circle(frame, center_i, 5, (0, 0, 255), -1)
                cv2.circle(frame, center_j, 5, (0, 0, 255), -1)

    cv2.putText(frame, f'Current People: {num_people}', (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    if num_people > threshold:
        cv2.putText(frame, f'WARNING: Crowd exceeds {threshold}!', (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        warning_shown = True
    else:
        warning_shown = False

    return frame, warning_shown

def detect_on_video(video_path, threshold=10):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Unable to open video file {video_path}")
        return

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_fps = int(cap.get(cv2.CAP_PROP_FPS))
    out_video_path = 'processed_video.mp4'
    out_video = cv2.VideoWriter(out_video_path, cv2.VideoWriter_fourcc(*'mp4v'), frame_fps, (frame_width, frame_height))

    frame_count = 0
    detections = []
    track_colors = {}
    warning_shown = False

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        overlay, warning_shown = process_frame(frame, frame_count, detections, track_colors, threshold, warning_shown)
        out_video.write(overlay)
        frame_count += 1

    cap.release()
    out_video.release()

    print("Video processing complete. Download the processed video:")
    from google.colab import files
    files.download(out_video_path)

# Example usage
detect_on_video('crowd.mp4', threshold=40)


0: 384x640 41 Persons, 11.6ms
Speed: 1.9ms preprocess, 11.6ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 41 Persons, 10.5ms
Speed: 1.9ms preprocess, 10.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 40 Persons, 10.2ms
Speed: 2.3ms preprocess, 10.2ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 39 Persons, 9.8ms
Speed: 1.6ms preprocess, 9.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 37 Persons, 10.7ms
Speed: 1.9ms preprocess, 10.7ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 37 Persons, 9.8ms
Speed: 1.8ms preprocess, 9.8ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 38 Persons, 11.8ms
Speed: 2.2ms preprocess, 11.8ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 38 Persons, 11.5ms
Speed: 2.1ms preprocess, 11.5ms inference, 1.6ms postprocess per image at sha

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>